In [ ]:
from elasticsearch import Elasticsearch
from getpass import getpass  # For securely getting user input
from elasticsearch.client import MlClient

# Prompt the user to enter their Elastic Cloud ID and API Key securely
ELASTIC_CLOUD_ID = getpass("Elastic Cloud ID: ")
ELASTIC_API_KEY = getpass("Elastic API Key: ")

# Create an Elasticsearch client using the provided credentials
client = Elasticsearch(
    cloud_id=ELASTIC_CLOUD_ID,  # cloud id can be found under deployment management
    api_key=ELASTIC_API_KEY # API keys can be generated under management / security
)

client.info()

In [ ]:
index = "hp_scripts_final"
model_id = "sentence-transformers__msmarco-minilm-l-12-v3"
question = "magic spell"

def semantic_search(question):
    question = {"text_field" : question}
    result = MlClient.infer_trained_model(client, model_id =model_id, docs = question)
    query_vector = result["inference_results"][0]["predicted_value"]

    query = {
        "field": "text_embedding.predicted_value",
        "query_vector": query_vector,
        "k": 5,
        "num_candidates": 100
    }
        
    result = client.search(index = index, knn=query, source=["Sentence", "Character"])
    answer = []
    for element in result["hits"]["hits"]:
        answer.append("{}: {}, score {}".format(element["_source"]["Character"], element["_source"]["Sentence"], element["_score"]))

    return answer

In [ ]:
answer = semantic_search(question)
for element in answer:
    print(element)

In [ ]:
index = "historical_searches"

settings = {}
mappings = {
    "_meta" : {
        "created_by" : "Iulia Feroli"
    },
    "properties" : {
        "Query" : {
            "type" : "text"
        },
        "Reponse" : {
            "type" : "text"
        },
        "date" : {
            "type" : "date"
        }
    }
}

client.indices.create(index=index, settings=settings, mappings=mappings)

In [ ]:
from datetime import datetime

example = {"Query" : "I am searching", "Response" : ["first", "second"], "date" : datetime.now()}
response = client.index(index = index, document = example)
print(response['result'])

In [ ]:
response = client.search(index = index, sort=[{"date" : {"order": "desc"}}])
for hit in response["hits"]["hits"]:
    print(hit['_source'])